<a href="https://colab.research.google.com/github/naoya1110/road_following_by_regression/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Following by Regression - Train Model

## Introduction
Once you have finished data collection for you road following task, you need to train a model with your dataset.

### GPU
Please make sure we can use a GPU for model training.

In [ ]:
import torch

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)

### Basic Python Packages
Let's import basic Python packages.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm.notebook import tqdm
import shutil
import glob
import PIL.Image

## Data Preparation

### Upload & Extract Dataset

We need to upload the `dataset_reg.zip` file in the file browser of the Google Colab. This takes time depending the size of your dataset. After uploading is finished, you can extract it with the next command.

In [ ]:
! unzip -q dataset_reg.zip

Now we should see a directory named ``dataset_reg`` in the file browser.

#### *If your `dataset_reg.zip` file is in your google drive*

First you need to mount your google drive with the next command.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Then extract the `dataset_reg.zip` file. Remenber to change the file path appropriate for your `dataset_reg.zip`.

In [ ]:
!unzip -q /path/to/your/dataset_reg.zip

### Remove Broken Data
The dataset directory may contain some broken image data or unnecessary directories for some reasons. Such data will cause errors when we train the model. So it is better to remove those data before training.

In [ ]:
dataset_dir = "dataset_reg"
filenames = sorted(os.listdir(dataset_dir))

for filename in filenames:
    path = os.path.join(dataset_dir, filename)
    img = cv2.imread(path)
    if img is None:
        try:
            os.remove(path)
            print("Removed", path)
        except IsADirectoryError:
            shutil.rmtree(path)
            print("Removed", path)

### Create Dataset
Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [ ]:
import torch
import torchvision.transforms as transforms

def get_x(path, width):
    """Gets the x value from the image filename"""
    return float(int(path.split("_")[3])) / width

def get_y(path, height):
    """Gets the y value from the image filename"""
    return float(int(path.split("_")[4])) / height

means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

class RegDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))

        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, means, stds)
        
        return image, torch.tensor([x, y]).float()
    
dataset = RegDataset('dataset_reg')
print("Number of data:", len(dataset))

### Data Distribution
Let's visualize the distribution of x and y data.

In [ ]:
x_list = []
y_list = []
for i in range(len(dataset)):
    _, xy = dataset[i]
    x_list.append(xy[0])
    y_list.append(xy[1])

sns.jointplot(np.array(x_list), np.array(y_list), xlim=(0, 1), ylim=(1, 0))

### Train Test Split
We need to split the dataset for training and testing.

In [ ]:
test_size = 200
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

### Data Loaders
Then we can make data loaders for the training and testing datasets.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=50,
    shuffle=True,
    num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=50,
    shuffle=False,
    num_workers=2
)

## Model Building

### Model Architecture
We build a relatively simple CNN model but you can use any models you want.

In [ ]:
import torch.nn as nn

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),    # dropout layer
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)  
        return x
    
model = Model().to(device)
print(model)

### torchinfo

In [ ]:
! pip install torchinfo
from torchinfo import summary

for x_batch, _ in train_loader:
    break

input_shape = x_batch.shape
print(input_shape)

summary(model, input_shape)

## Training

### Training Loop

Now we can train the model. Since this is a regresion model we are going to use `MSELoss()` as the loss function.

In [ ]:
import torch.optim as optim

model = Model()
model = model.to(device)

loss_func = nn.MSELoss()                      # use MSE loss function.
optimizer = optim.Adam(model.parameters(), lr=1E-4)    # set optimizer
epochs = 30

best_model_path = 'best_model_reg.pth'
best_loss = 1E+9

# create empty lists for saving metrics during training
train_loss_list = []
test_loss_list = []

for epoch in range(epochs):
    print("-----------------------------")
    print(f"Epoch {epoch+1}/{epochs}")

    # initialize metrics
    train_loss = 0
    test_loss = 0

    #--- Training Phase ---#
    model.train()    # set model to training mode

    pbar = tqdm(train_loader)
    pbar.set_description("Train")

    for x_batch, y_batch in pbar:      # take mini batch data from train_loader
        
        x_batch = x_batch.to(device)     # load x_batch data on GPU
        y_batch = y_batch.to(device)     # load y_batch data on GPU
    
        optimizer.zero_grad()                  # reset gradients to 0
        p_batch = model(x_batch)               # do prediction
        loss = loss_func(p_batch, y_batch)     # measure loss
        loss.backward()                        # calculate gradients
        optimizer.step()                       # update model parameters

        train_loss += loss.item()                                # accumulate loss value

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})
    #----------------------#

    #--- Evaluation Phase ---#
    with torch.no_grad():   # disable autograd for saving memory usage
        model.eval()        # set model to evaluation mode

        pbar = tqdm(test_loader)
        pbar.set_description("Test") 

        for x_batch, y_batch in pbar:   # take mini batch data from test_loader

            x_batch = x_batch.to(device)     # load x_batch data on GPU
            y_batch = y_batch.to(device)     # load y_batch data on GPU

            p_batch = model(x_batch)              # do prediction
            loss = loss_func(p_batch, y_batch)    # measure loss

            test_loss += loss.item()                                # accumulate loss value
            p_batch_label = torch.argmax(p_batch, dim=1)            # convert p_batch vector to p_batch_label

            pbar.set_postfix({"loss": f"{loss.item():.4f}"})
    #------------------------#

    train_loss = train_loss/len(train_loader)                 # determine loss for training data
    test_loss = test_loss/len(test_loader)                    # determine loss for test data 

    # show and store metrics
    print(f"Train Loss={train_loss:.5f}, Test Loss={test_loss:.5f}")

    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

    # save the model if test accuracy is better than before
    if test_loss < best_loss:
        torch.save(model.state_dict(), best_model_path)
        print(f"Test loss decreased from {best_loss:.3f} to {test_loss:.3f}")
        print(f"Model saved at {best_model_path}")
        best_loss = test_loss

### Learning Curves
Let's check the learning curves.

In [ ]:
plt.rcParams["font.size"]=14
real_epochs = np.arange(len(train_loss_list))+1

plt.plot(real_epochs, train_loss_list, c="#1f77b4", label="train loss")
plt.plot(real_epochs, test_loss_list, lw=0, marker="o", c="#1f77b4", label="test loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.legend()

## Evaluation

### Load The Best Model
Let's load the best model parameters.

In [ ]:
model.load_state_dict(torch.load(best_model_path))

In [ ]:
truths = []
preds = []

with torch.no_grad():   # disable autograd for saving memory usage
    model.eval()        # set model to evaluation mode

    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)     # load x_batch data on GPU
        p_batch = model(x_batch)    # predict

        p_batch = p_batch.to("cpu").numpy()
        y_batch = y_batch.to("cpu").numpy()
        preds.append(p_batch)
        truths.append(y_batch)

preds = np.array(preds).reshape(test_size, 2)
truths = np.array(truths).reshape(test_size, 2)

plt.figure(figsize=(9, 4))

plt.subplot(1, 2, 1)
plt.plot([-1.2, 1.2], [-1.2, 1.2])
plt.plot(truths[:,0], preds[:,0], lw=0, marker="o")
plt.grid()
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.xlabel("Ground Truth")
plt.ylabel("Prediction")
plt.title("x position")

plt.subplot(1, 2, 2)
plt.plot([-1.2, 1.2], [-1.2, 1.2])
plt.plot(truths[:,1], preds[:,1], lw=0, marker="o")
plt.grid()
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.xlabel("Ground Truth")
plt.title("y position")

### Predictions for Test Data
Let's see the predictions for the test dataset.

In [ ]:
plt.figure(figsize=(20, 14))

for i in range(50):
    image, xy = test_dataset[i]
    image = np.transpose(image, (1,2,0))
    
    image[:, :, 0] = image[:, :, 0]*stds[0]+means[0]
    image[:, :, 1] = image[:, :, 1]*stds[1]+means[1]
    image[:, :, 2] = image[:, :, 2]*stds[2]+means[2]
    image = (255*image).numpy().astype("uint8")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    xy_truth = tuple((truths[i]*224).astype("int"))
    xy_pred = tuple((preds[i]*224).astype("int"))

    plt.subplot(5, 10, i+1)
    image = cv2.circle(image, xy_truth, 8, (0,255, 0), 3)
    image = cv2.circle(image, xy_pred, 8, (255, 0, 255), 3)
    plt.imshow(image)
    plt.axis("off")

## Download Model
Once you finished training, download `best_model_reg.pth` from the file browser and upload it to your JetBot.